In [1]:
import numpy as np

#import matplotlib.pyplot as plt
#from bokeh.io import push_notebook, show, output_notebook; 
#from bokeh.plotting import figure; 
#from bokeh.layouts import column
#output_notebook()

import USGShydro


to do:

- download rainfall
- download runoff
- create time series dataset of lags, etc.
- drop observations with missing values
- run rainfall-runoff model
- plot predicted runoff vs. actual
- predict days above flood stage with models trained over different time periods

-universalize the procedure for other sites
-- create tool to query sites for dates and availability of needed series
-compare sites and see if model relates to hydrologic featues, basin topography, landcover, etc.

In [2]:
site='sites=02314495'
start='startDT=1951-01-01T00:00%2b0000'
start='startDT=2019-01-01T00:00%2b0000'#short interval for practice
end='endDT=2019-04-19T06:56-0400'
#data1=getUSGS.Get_data(site, start, end)
paramlist=['00065', '000045'] #must be entered as strings
modelfeatures=[]
try1=USGShydro.Hydrositedatamodel(site,start,end,paramlist,modelfeatures)

a
b


In [3]:
try1.get_data()

https://waterservices.usgs.gov/nwis/iv/?format=waterml,2.0&sites=02314495&startDT=2019-01-01T00:00%2b0000&endDT=2019-04-19T06:56-0400&parameterCd=00065,000045&siteStatus=all
